In [1]:
from collections import defaultdict
import numpy as np
import math

In [2]:
import nltk
nltk.download('universal_tagset')
nltk.download('brown')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\vinay\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\vinay\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
# To perform Laplace smoothing
def laplace_smoothing(num,den):
    return (num+0.001)/(den+0.001)

In [4]:
# Calculates initial state distribution
def initial_state_dis(corpus):
    count_POS =defaultdict(int)
    total_count=len(corpus)
    for i in corpus:
        count_POS[i[0][1]]+=1
    for i in count_POS:
        count_POS[i]= laplace_smoothing(count_POS[i],total_count)
    return count_POS

In [5]:
# gets a list of POS occuring in the training dat
def get_all_POS(corpus):
    all_POS=[]
    for i in corpus:
        for j in i:
            all_POS.append(j[1])
    all_POS=list(set(all_POS))
    return all_POS

In [6]:
# Maps every POS to a number so as to be used in Transition and Observation matrices
def map_index(all_POS):
    map_index ={}
    for idx,i in enumerate(all_POS):
        map_index[i]=idx
    return map_index

In [7]:
# Creates the Transition Matrix
def transition_matrix(corpus,map_pos_idx):
    num_pos = len(map_pos_idx)
    transition_mat = [[0 for j in range(num_pos)] for i in range(num_pos)]
    for i in corpus:
        for j in range(len(i)-1):
            transition_mat[map_pos_idx[i[j][1]]][map_pos_idx[i[j+1][1]]]+=1
    for i in range(num_pos):
        current_len=sum(transition_mat[i])
        for j in range(num_pos):
            transition_mat[i][j] = laplace_smoothing(transition_mat[i][j],current_len)
    return np.array(transition_mat)
    

In [8]:
# Creates the vocabulary of words taking OOV into account
def create_Vocab(corpus):
    vocab_count =defaultdict(int)
    for i in corpus:
        for j in i:
            vocab_count[j[0].lower()]+=1
    new_vocab_count=defaultdict(int)
    for i in vocab_count:
        if(vocab_count[i]>=5):
            new_vocab_count[i]=vocab_count[i]
    return ['OOV']+list(new_vocab_count.keys())

In [9]:
# Creates the observation matrix
def observation_matrix(corpus,map_pos_idx,vocab_index):
    rows = len(map_pos_idx)
    cols =len(vocab_index)
    observation_mat = [[0 for j in range(cols)] for i in range(rows)]
    for i in corpus:
        for j in i:
            if j[0].lower() in vocab_index:
                observation_mat[map_pos_idx[j[1]]][vocab_index[j[0].lower()]]+=1
            else:
                observation_mat[map_pos_idx[j[1]]][0]+=1
    for i in range(rows):
        current_len=sum(observation_mat[i])
        for j in range(cols):
            observation_mat[i][j] = laplace_smoothing(observation_mat[i][j],current_len)
    return np.array(observation_mat)

In [10]:
# Maps the initial state distriution to an index
def createInitialMap(initial_state,all_POS):
    return [initial_state[i] for i in all_POS]

In [11]:
# Convert observation to appropriate index in the vocabulary
def createobs(test_data,vocab_index):
    row=len(test_data)
    obs =[]
    for i in test_data:
        cur_test=[]
        for j in i:
            if j[0].lower() in vocab_index:
                cur_test.append(vocab_index[j[0].lower()])
            else:
                cur_test.append(0)
        obs.append(cur_test)
    return obs
            

**Corpus**

In [12]:
corpus=nltk.corpus.brown.tagged_sents(tagset='universal')[:10000]
corpus

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

**Initial state distribution**

In [13]:
initial_state= initial_state_dis(corpus)
initial_state

defaultdict(int,
            {'DET': 0.24030007596999242,
             '.': 0.06500009349999064,
             'PRON': 0.11340008865999113,
             'NOUN': 0.19650008034999195,
             'ADV': 0.07750009224999077,
             'ADP': 0.1207000879299912,
             'VERB': 0.04100009589999041,
             'NUM': 0.017300098269990172,
             'CONJ': 0.054900094509990546,
             'ADJ': 0.04220009577999042,
             'PRT': 0.030700096929990303,
             'X': 0.000500099949990005})

In [14]:
all_POS=get_all_POS(corpus)
map_pos_idx=map_index(all_POS)

**Total Part of speech**

In [15]:
all_POS

['DET',
 'ADJ',
 '.',
 'NOUN',
 'PRT',
 'NUM',
 'X',
 'CONJ',
 'VERB',
 'PRON',
 'ADV',
 'ADP']

In [16]:
map_pos_idx

{'DET': 0,
 'ADJ': 1,
 '.': 2,
 'NOUN': 3,
 'PRT': 4,
 'NUM': 5,
 'X': 6,
 'CONJ': 7,
 'VERB': 8,
 'PRON': 9,
 'ADV': 10,
 'ADP': 11}

**Transition matrix**

In [17]:
A=transition_matrix(corpus,map_pos_idx)
A

array([[6.43078157e-03, 2.50370284e-01, 1.29004984e-02, 6.15636464e-01,
        1.55900688e-03, 1.33681887e-02, 1.52003268e-03, 5.06703542e-04,
        6.14623485e-02, 8.37949145e-03, 1.88635506e-02, 9.00307861e-03],
       [6.03480609e-03, 5.92502859e-02, 8.94240116e-02, 6.74855247e-01,
        1.85309955e-02, 1.28010355e-02, 6.09631173e-04, 3.32825947e-02,
        1.54831444e-02, 2.37738480e-03, 7.61968865e-03, 7.97318452e-02],
       [1.20139515e-01, 5.28306906e-02, 1.41649974e-01, 1.78997803e-01,
        2.01513403e-02, 2.13332336e-02, 1.95018308e-03, 9.38423890e-02,
        1.30776555e-01, 6.38813932e-02, 6.30540679e-02, 1.11393505e-01],
       [1.41032272e-02, 1.56665151e-02, 2.64632724e-01, 2.12203745e-01,
        1.74651367e-02, 9.69912574e-03, 4.70684141e-04, 5.25130433e-02,
        1.44024220e-01, 1.77340895e-02, 2.28105728e-02, 2.28677102e-01],
       [8.46823757e-02, 1.89500555e-02, 4.65852652e-02, 4.14530120e-02,
        8.68554902e-03, 7.10639418e-03, 1.97591749e-04, 7.89

**Vocabulary**

In [18]:
vocab=create_Vocab(corpus)
vocab

['OOV',
 'the',
 'fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'an',
 'investigation',
 'of',
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'no',
 'evidence',
 "''",
 'that',
 'any',
 'took',
 'place',
 '.',
 'further',
 'in',
 'city',
 'executive',
 'committee',
 ',',
 'which',
 'had',
 'over-all',
 'charge',
 'deserves',
 'praise',
 'and',
 'thanks',
 'atlanta',
 'for',
 'manner',
 'was',
 'conducted',
 'term',
 'been',
 'charged',
 'by',
 'superior',
 'court',
 'judge',
 'to',
 'reports',
 'possible',
 'won',
 'allen',
 'jr.',
 'only',
 'a',
 'relative',
 'such',
 'received',
 'considering',
 'widespread',
 'interest',
 'number',
 'voters',
 'size',
 'this',
 'it',
 'did',
 'find',
 'many',
 "georgia's",
 'laws',
 'are',
 'or',
 'inadequate',
 'often',
 'recommended',
 'legislators',
 'act',
 'have',
 'these',
 'studied',
 'revised',
 'end',
 'improving',
 'them',
 'commented',
 'on',
 'other',
 'among',
 'purchasing',
 'departments',
 'well',
 'follow',
 '

In [19]:
vocab_index = map_index(vocab)
vocab_index

{'OOV': 0,
 'the': 1,
 'fulton': 2,
 'county': 3,
 'grand': 4,
 'jury': 5,
 'said': 6,
 'friday': 7,
 'an': 8,
 'investigation': 9,
 'of': 10,
 'recent': 11,
 'primary': 12,
 'election': 13,
 'produced': 14,
 '``': 15,
 'no': 16,
 'evidence': 17,
 "''": 18,
 'that': 19,
 'any': 20,
 'took': 21,
 'place': 22,
 '.': 23,
 'further': 24,
 'in': 25,
 'city': 26,
 'executive': 27,
 'committee': 28,
 ',': 29,
 'which': 30,
 'had': 31,
 'over-all': 32,
 'charge': 33,
 'deserves': 34,
 'praise': 35,
 'and': 36,
 'thanks': 37,
 'atlanta': 38,
 'for': 39,
 'manner': 40,
 'was': 41,
 'conducted': 42,
 'term': 43,
 'been': 44,
 'charged': 45,
 'by': 46,
 'superior': 47,
 'court': 48,
 'judge': 49,
 'to': 50,
 'reports': 51,
 'possible': 52,
 'won': 53,
 'allen': 54,
 'jr.': 55,
 'only': 56,
 'a': 57,
 'relative': 58,
 'such': 59,
 'received': 60,
 'considering': 61,
 'widespread': 62,
 'interest': 63,
 'number': 64,
 'voters': 65,
 'size': 66,
 'this': 67,
 'it': 68,
 'did': 69,
 'find': 70,
 'many

**Observation Matrix**

In [20]:
B=observation_matrix(corpus,map_pos_idx,vocab_index)
B

array([[1.55917534e-04, 5.34040001e-01, 3.89696411e-08, ...,
        3.89696411e-08, 3.89696411e-08, 3.89696411e-08],
       [2.19969606e-01, 6.08827969e-08, 6.08827969e-08, ...,
        6.08827969e-08, 6.08827969e-08, 6.08827969e-08],
       [3.78229117e-08, 3.78229117e-08, 3.78229117e-08, ...,
        3.78229117e-08, 3.78229117e-08, 3.78229117e-08],
       ...,
       [2.88906502e-03, 1.44446029e-07, 1.44446029e-07, ...,
        1.44446029e-07, 1.44446029e-07, 1.44446029e-07],
       [8.97640426e-02, 1.05853699e-07, 1.05853699e-07, ...,
        1.05853699e-07, 1.05853699e-07, 1.05853699e-07],
       [1.32556422e-03, 3.68202056e-08, 3.68202056e-08, ...,
        3.68202056e-08, 3.68202056e-08, 3.68202056e-08]])

**Viterbi Algorithm**

In [21]:
def viterbi(obs,pi,A,B):
    rows=len(pi)
    cols=len(obs)
    dp=[[0 for j in range(cols)] for i in range(rows)]
    state = [[0 for j in range(cols)] for i in range(rows)]
    for j in range(cols):
        for i in range(rows-1,-1,-1):
            if(j==0):
                dp[i][j]= pi[i]*B[i][obs[j]]
                state[i][j]= 0
            else:
                max_val=float('-inf')
                max_state =0
                for k in range(rows):
                    val = math.log(dp[k][j-1])+math.log(A[k][i])+math.log(B[i][obs[j]])
                    #print(val)
                    if(val>max_val):
                        max_val=val
                        max_state=k
               # print('next')
                state[i][j]=max_state
                dp[i][j]=math.exp(max_val)
    #print(dp)
    #print(state)
    max_val_last=float('-inf')
    best=0
    for i in range(rows):
        if(max_val_last<dp[i][cols-1]):
            max_val_last = dp[i][cols-1]
            states=[i]
            best=state[i][cols-1]
    for j in range(cols-2,-1,-1):
        states= [best]+states
        best = state[best][j]
        
    return states

**Inference**

In [22]:
test_data=nltk.corpus.brown.tagged_sents(tagset='universal')[10150:10153]
test_data

[[('Those', 'DET'),
  ('coming', 'VERB'),
  ('from', 'ADP'),
  ('other', 'ADJ'),
  ('denominations', 'NOUN'),
  ('will', 'VERB'),
  ('welcome', 'VERB'),
  ('the', 'DET'),
  ('opportunity', 'NOUN'),
  ('to', 'PRT'),
  ('become', 'VERB'),
  ('informed', 'VERB'),
  ('.', '.')],
 [('The', 'DET'),
  ('preparatory', 'ADJ'),
  ('class', 'NOUN'),
  ('is', 'VERB'),
  ('an', 'DET'),
  ('introductory', 'ADJ'),
  ('face-to-face', 'ADJ'),
  ('group', 'NOUN'),
  ('in', 'ADP'),
  ('which', 'DET'),
  ('new', 'ADJ'),
  ('members', 'NOUN'),
  ('become', 'VERB'),
  ('acquainted', 'VERB'),
  ('with', 'ADP'),
  ('one', 'NUM'),
  ('another', 'DET'),
  ('.', '.')],
 [('It', 'PRON'),
  ('provides', 'VERB'),
  ('a', 'DET'),
  ('natural', 'ADJ'),
  ('transition', 'NOUN'),
  ('into', 'ADP'),
  ('the', 'DET'),
  ('life', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('local', 'ADJ'),
  ('church', 'NOUN'),
  ('and', 'CONJ'),
  ('its', 'DET'),
  ('organizations', 'NOUN'),
  ('.', '.')]]

In [23]:
pi=createInitialMap(initial_state,all_POS)

In [24]:
obs =createobs(test_data,vocab_index)
obs

[[1200, 779, 210, 90, 0, 185, 3257, 1, 2017, 50, 429, 799, 23],
 [1, 0, 3086, 117, 8, 0, 0, 842, 25, 30, 216, 561, 429, 0, 164, 151, 825, 23],
 [68, 2095, 57, 858, 2296, 197, 1, 1892, 10, 1, 637, 604, 36, 191, 4157, 23]]

In [25]:
# Call the vitrbi algorithm for every test example
predicted =[]
for i in obs:
    result=viterbi(i,pi,A,B)
    sent_pred=[]
    for j in result:
        sent_pred.append(all_POS[j])
    predicted.append(sent_pred)

In [26]:
# Print the output in appropriate format
for idx,i in enumerate(test_data):
    actual_text=[]
    actual_pos=[]
    for j in i:
        actual_text.append(j[0])
        actual_pos.append(j[1])
    print("Actual Text")
    print(actual_text)
    print("Actual POS")
    print(actual_pos)
    print("Predicted POS")
    print(predicted[idx])
    print("")

Actual Text
['Those', 'coming', 'from', 'other', 'denominations', 'will', 'welcome', 'the', 'opportunity', 'to', 'become', 'informed', '.']
Actual POS
['DET', 'VERB', 'ADP', 'ADJ', 'NOUN', 'VERB', 'VERB', 'DET', 'NOUN', 'PRT', 'VERB', 'VERB', '.']
Predicted POS
['DET', 'VERB', 'ADP', 'ADJ', 'NOUN', 'VERB', 'VERB', 'DET', 'NOUN', 'PRT', 'VERB', 'VERB', '.']

Actual Text
['The', 'preparatory', 'class', 'is', 'an', 'introductory', 'face-to-face', 'group', 'in', 'which', 'new', 'members', 'become', 'acquainted', 'with', 'one', 'another', '.']
Actual POS
['DET', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'ADP', 'NUM', 'DET', '.']
Predicted POS
['DET', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'ADP', 'NUM', 'DET', '.']

Actual Text
['It', 'provides', 'a', 'natural', 'transition', 'into', 'the', 'life', 'of', 'the', 'local', 'church', 'and', 'its', 'organizations', '.']
Actual POS
['P